<a href="https://colab.research.google.com/github/skyshine460/Systematic_Review/blob/main/Pytorch_BERT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch と BERT を使ったベースライン（titleのみ）

In [1]:
import os
import sys

from google.colab import drive
drive.mount('/gdrive')

!cp /gdrive/MyDrive/Datasets/signate-471/train.csv .
!cp /gdrive/MyDrive/Datasets/signate-471/test.csv .
!cp /gdrive/MyDrive/Datasets/signate-471/sample_submit.csv .

Mounted at /gdrive


In [2]:
!pip install -q transformers

In [3]:
import math
import random
import time
import warnings

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers as T
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

In [4]:
DATA_DIR = "./"
OUTPUT_DIR = "./"

warnings.filterwarnings("ignore")

In [5]:
# GPUを優先利用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# ログを出力する関数
def init_logger(log_file=OUTPUT_DIR + "train.log"):
    from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

    # ロガーを作成または取得（スクリプトの名前を使用）
    logger = getLogger(__name__)
    logger.setLevel(INFO)  # ログレベルをINFOに設定（INFO以上のログが出力される）

    # 標準出力用のハンドラを作成
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))  # メッセージのみを出力するフォーマット

    # ファイル出力用のハンドラを作成
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))  # メッセージのみを出力するフォーマット

    # ロガーにハンドラを追加（標準出力 + ファイル出力）
    logger.addHandler(handler1)
    logger.addHandler(handler2)

    return logger  # 初期化したロガーを返す

# ロガーを初期化
LOGGER = init_logger()

In [7]:
# ランダムシードを固定
def seed_torch(seed=42):
    random.seed(seed)    # Python 標準ライブラリ
    os.environ["PYTHONHASHSEED"] = str(seed)    # Python 環境変数 "PYTHONHASHSEED" を設定（ハッシュ関連の乱数を固定）
    np.random.seed(seed)    # NumPy
    torch.manual_seed(seed)    # PyTorch（CPU用）
    torch.cuda.manual_seed(seed)    # PyTorch（GPU用）
    torch.cuda.manual_seed_all(seed)    # PyTorch（マルチGPU用）
    torch.backends.cudnn.deterministic = True    # PyTorch の cuDNN を再現性のあるモードに設定（計算の決定性を保証）

# シード値の設定
seed = 471
seed_torch(seed)


In [8]:
# データセットの読み込み
train = pd.read_csv(DATA_DIR + "train.csv")
test = pd.read_csv(DATA_DIR + "test.csv")
sub = pd.read_csv(DATA_DIR + "sample_submit.csv", header=None)
sub.columns = ["id", "judgement"]

In [9]:
# pos/neg の割合
# trainデータとtestデータの分布を確認
# この値を境に、モデルの出力を 0 と 1 にします。→参考値。適切な値を検討
border = len(train[train["judgement"] == 1]) / len(train["judgement"])
print(border)

0.023282372444280715


In [10]:
# 交差検証用の番号をデータに付与する関数
def get_train_data(train):
    # StratifiedKFoldを使用して交差検証用の分割を作成
    Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    # 交差検証の分割に基づき、各行に "fold" 番号を付与
    for n, (train_index, val_index) in enumerate(Fold.split(train, train["judgement"])):
        train.loc[val_index, "fold"] = int(n)  # 検証用のインデックスに fold 番号を振る

    # "fold" 列を符号なし8ビット整数型に変換（メモリ効率の向上）
    train["fold"] = train["fold"].astype(np.uint8)

    return train  # fold列が追加されたデータを返す

In [11]:
# データをそのまま渡す関数
def get_test_data(test):
    return test

train = get_train_data(train)

In [12]:
# PyTorch Datasetクラスを継承したカスタムデータセットクラス。
# テキストデータをBERT用にトークン化し、学習や推論に適した形式に整形。
class BaseDataset(Dataset):

    # BERT用のデータ準備
    def __init__(self, df, model_name, include_labels=True):

        # 指定したBERTモデルのトークナイザをロード
        tokenizer = T.BertTokenizer.from_pretrained(model_name)

        self.df = df  # データフレームを保持
        self.include_labels = include_labels  # ラベルを含むかどうかを保持

        # 'title' カラムのテキストデータをリストに変換
        self.title = df["title"].tolist()

        # BERTトークナイザでバッチエンコード
        self.encoded = tokenizer.batch_encode_plus(
            self.title,
            padding='max_length',             # 最大長に合わせてパディングを追加
            max_length=72,                    # 最大長を72トークンに設定
            truncation=True,                  # 最大長を超える場合は切り詰め
            return_attention_mask=True        # Attention Maskも返す
        )

        # ラベルを含む場合は 'judgement' カラムからラベルを取得
        if self.include_labels:
            self.labels = df["judgement"].values

    # データフレームの行数を返す関数
    def __len__(self):
        return len(self.df)

    # 指定されたインデックスのデータを取得。
    def __getitem__(self, idx):

        # トークナイズされた input_ids と attention_mask を取得
        input_ids = torch.tensor(self.encoded['input_ids'][idx])  # トークンID列
        attention_mask = torch.tensor(self.encoded['attention_mask'][idx])  # Attention Mask列

        # ラベルを含む場合はラベルも返す
        if self.include_labels:
            label = torch.tensor(self.labels[idx]).float()  # ラベル値をテンソルに変換
            return input_ids, attention_mask, label

        # ラベルが不要な場合は input_ids と attention_mask のみ返す
        return input_ids, attention_mask

In [13]:
# BERTモデルをベースとしたPyTorchのカスタムモデルクラス。
class BaseModel(nn.Module):

    # 指定されたBERTモデルをロードし、シーケンス分類用に設定。
    def __init__(self, model_name):

        super().__init__()  # nn.Moduleの初期化メソッドを呼び出し

        # Hugging FaceのBERTモデル（Sequence Classification用）をロード
        self.model = T.BertForSequenceClassification.from_pretrained(
            model_name,  # モデル名（事前学習済みBERTの名前）
            num_labels=1  # 出力ラベル数。1に設定しているので回帰/二値分類用。
        )

        # 出力にシグモイド関数を適用して確率を返す設定
        self.sigmoid = nn.Sigmoid()

    # 順伝播メソッド。モデルの計算グラフを定義。
    def forward(self, input_ids, attention_mask):

        # BERTモデルの順伝播。logits（未スケールの出力）を取得
        out = self.model(input_ids=input_ids, attention_mask=attention_mask)

        # ロジットにシグモイド関数を適用して確率値に変換
        out = self.sigmoid(out.logits).squeeze()  # squeezeで次元を整形

        return out  # モデルの出力を返す


In [14]:
# 値の平均と現在の値を計算・保持するためのクラス。
class AverageMeter(object):

    # 初期化メソッド。
    def __init__(self):
        self.reset()

    # 値をリセット（初期化）する。
    def reset(self):
        self.val = 0    # 現在の値
        self.avg = 0    # 平均値
        self.sum = 0    # 合計値
        self.count = 0  # 値の更新回数

    # 値を更新し、平均値を計算する。
    def update(self, val, n=1):
        self.val = val                  # 現在の値を更新
        self.sum += val * n             # 合計値を更新
        self.count += n                 # 更新回数を増加
        self.avg = self.sum / self.count  # 平均値を計算

# 秒単位の時間を「分:秒」の形式に変換する。
def asMinutes(s):
    m = math.floor(s / 60)  # 分を計算
    s -= m * 60             # 秒を計算（余り）
    return "%dm %ds" % (m, s)

# 経過時間と残り時間を計算し、「経過時間（残り時間）」の形式で返す。
def timeSince(since, percent):
    now = time.time()  # 現在時刻を取得
    s = now - since    # 経過時間を計算
    es = s / percent   # 処理全体にかかる時間を推定
    rs = es - s        # 残り時間を計算
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

In [15]:
# モデルを1エポック分学習する関数。
def train_fn(train_loader, model, criterion, optimizer, epoch, device):
    """
    Args:
        train_loader (DataLoader): 学習データのデータローダー。
        model (nn.Module): 学習対象のPyTorchモデル。
        criterion (nn.Module): 損失関数（例: nn.CrossEntropyLoss）。
        optimizer (torch.optim.Optimizer): 最適化アルゴリズム（例: Adam, SGD）。
        epoch (int): 現在のエポック数（0から開始）。
        device (torch.device): 計算に使用するデバイス（例: 'cuda' or 'cpu'）。

    Returns:
        float: エポック中の平均損失値。
    """
    start = end = time.time()  # 学習の開始時間を記録
    losses = AverageMeter()  # 平均損失値を記録するためのヘルパークラス

    # モデルを学習モードに切り替え（DropoutやBatchNormを有効化）
    model.train()

    # ミニバッチ学習を実行
    for step, (input_ids, attention_mask, labels) in enumerate(train_loader):
        optimizer.zero_grad()  # 勾配を初期化

        # データをデバイス（CPU/GPU）に転送
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)  # 現在のバッチサイズ

        # モデルの順伝播を実行
        y_preds = model(input_ids, attention_mask)

        # 損失を計算
        loss = criterion(y_preds, labels)

        # 平均損失を更新
        losses.update(loss.item(), batch_size)

        # 逆伝播を実行して勾配を計算
        loss.backward()

        # パラメータを更新
        optimizer.step()

        # ログ出力: 進捗状況や損失を表示
        if step % 100 == 0 or step == (len(train_loader) - 1):
            print(
                f"Epoch: [{epoch + 1}][{step}/{len(train_loader)}] "  # エポックと現在のステップ
                f"Elapsed {timeSince(start, float(step + 1) / len(train_loader)):s} "  # 経過時間と残り時間
                f"Loss: {losses.avg:.4f} "  # 現在の平均損失
            )

    # エポック全体の平均損失値を返す
    return losses.avg

In [16]:
# モデルの検証を行う関数。
def valid_fn(valid_loader, model, criterion, device):
    """
    Args:
        valid_loader (DataLoader): 検証データのデータローダー。
        model (nn.Module): 検証対象のPyTorchモデル。
        criterion (nn.Module): 損失関数（例: nn.CrossEntropyLoss）。
        device (torch.device): 計算に使用するデバイス（例: 'cuda' or 'cpu'）。

    Returns:
        tuple: 平均損失値（float）、検証データ全体の予測値（numpy配列）。
    """
    start = end = time.time()  # 検証開始時刻を記録
    losses = AverageMeter()  # 平均損失値を記録するためのヘルパークラス

    # モデルを評価モードに切り替え（DropoutやBatchNormを無効化）
    model.eval()
    preds = []  # 検証データ全体の予測値を保持するリスト

    # 検証データをバッチごとに処理
    for step, (input_ids, attention_mask, labels) in enumerate(valid_loader):
        # データをデバイス（CPU/GPU）に転送
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)  # 現在のバッチサイズ

        # 損失を計算（勾配計算は不要）
        with torch.no_grad():  # 検証時は勾配計算を無効化してメモリ効率を向上
            y_preds = model(input_ids, attention_mask)

        # 損失を計算
        loss = criterion(y_preds, labels)

        # 平均損失を更新
        losses.update(loss.item(), batch_size)

        # 予測結果を記録（CPUに転送してnumpy配列に変換）
        preds.append(y_preds.to("cpu").numpy())

        # 進捗状況をログ出力
        if step % 100 == 0 or step == (len(valid_loader) - 1):
            print(
                f"EVAL: [{step}/{len(valid_loader)}] "  # 現在のステップと全体のステップ数
                f"Elapsed {timeSince(start, float(step + 1) / len(valid_loader)):s} "  # 経過時間と残り時間
                f"Loss: {losses.avg:.4f} "  # 現在の平均損失
            )

    # 検証データ全体の予測値を1つの配列に結合
    predictions = np.concatenate(preds)

    # 平均損失値と予測値を返す
    return losses.avg, predictions


In [17]:
# 学習済みモデルを使用してテストデータの推論を行う関数。
def inference():

    predictions = []  # 各foldモデルの予測結果を格納するリスト

    # テストデータセットの準備（ラベルは含まない設定）
    test_dataset = BaseDataset(test, "bert-base-uncased", include_labels=False)

    # テストデータローダーの設定
    test_loader = DataLoader(
        test_dataset,  # テスト用データセット
        batch_size=16,  # バッチサイズ
        shuffle=False,  # 順序をシャッフルしない（順番通りのデータ取得）
        num_workers=4,  # データローディングのための並列処理数
        pin_memory=True  # メモリ固定（GPUの高速化に有効）
    )

    # 5つのfoldモデルで推論を行う
    for fold in range(5):
        LOGGER.info(f"========== model: bert-base-uncased fold: {fold} inference ==========")
        # モデルの構築
        model = BaseModel("bert-base-uncased")
        model.to(device)  # モデルをデバイス（GPU/CPU）に転送

        # 学習済みモデルの重みをロード
        model.load_state_dict(
            torch.load(OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth")["model"]
        )
        model.eval()  # モデルを評価モードに切り替え

        preds = []  # 現在のfoldモデルの予測結果を格納するリスト

        # テストデータローダーからデータをバッチ単位で取得して推論
        for i, (input_ids, attention_mask) in tqdm(enumerate(test_loader), total=len(test_loader)):
            # 入力データをデバイスに転送
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            # 推論（勾配計算は不要）
            with torch.no_grad():
                y_preds = model(input_ids, attention_mask)

            # CPUに戻してnumpy配列に変換し、リストに追加
            preds.append(y_preds.to("cpu").numpy())

        # 現在のfoldモデルの全バッチの予測結果を結合
        preds = np.concatenate(preds)

        # foldモデルの予測結果を全体のリストに追加
        predictions.append(preds)

    # 各foldモデルの予測値を平均化（アンサンブル）
    predictions = np.mean(predictions, axis=0)

    return predictions  # 最終予測値を返す

In [18]:
# 指定したフォールドでモデルの学習と評価を行う関数。
def train_loop(train, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # Data Loader (データローダーの準備)
    # ====================================================
    # 学習用データと検証用データに分割
    trn_idx = train[train["fold"] != fold].index  # 現在のフォールド以外を学習用データに
    val_idx = train[train["fold"] == fold].index  # 現在のフォールドを検証用データに

    train_folds = train.loc[trn_idx].reset_index(drop=True)  # 学習データを取得
    valid_folds = train.loc[val_idx].reset_index(drop=True)  # 検証データを取得

    # データセットを作成
    train_dataset = BaseDataset(train_folds, "bert-base-uncased")
    valid_dataset = BaseDataset(valid_folds, "bert-base-uncased")

    # データローダーを作成
    train_loader = DataLoader(
        train_dataset,
        batch_size=16,  # バッチサイズ
        shuffle=True,  # 学習時はデータをシャッフル
        num_workers=4,  # データのロードに使用する並列ワーカー数
        pin_memory=True,  # メモリ固定（GPU高速化）
        drop_last=True,  # 最後の不完全なバッチを破棄
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=16,  # バッチサイズ
        shuffle=False,  # 検証時はシャッフルしない
        num_workers=4,
        pin_memory=True,
        drop_last=False,  # 不完全なバッチも使用
    )

    # ====================================================
    # Model (モデルの構築)
    # ====================================================
    model = BaseModel("bert-base-uncased")  # BERTベースのモデルを構築
    model.to(device)  # モデルをデバイス（CPU/GPU）に転送

    optimizer = T.AdamW(model.parameters(), lr=2e-5)  # AdamWオプティマイザ
    criterion = nn.BCELoss()  # バイナリクロスエントロピー損失関数

    # ====================================================
    # Loop (学習ループ)
    # ====================================================
    best_score = -1  # 最良スコアの初期値
    best_loss = np.inf  # 最良損失の初期値（無限大）

    # エポックごとに学習と評価を実行
    for epoch in range(3):
        start_time = time.time()  # エポック開始時刻を記録

        # 学習
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, device)

        # 検証
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        valid_labels = valid_folds["judgement"].values  # 検証データのラベルを取得

        # スコアの計算
        score = fbeta_score(valid_labels, np.where(preds < border, 0, 1), beta=7.0)

        # エポックごとのログを出力
        elapsed = time.time() - start_time  # エポックごとの経過時間
        LOGGER.info(
            f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
        )
        LOGGER.info(f"Epoch {epoch+1} - Score: {score}")

        # 最良スコアを更新した場合、モデルを保存
        if score > best_score:
            best_score = score
            LOGGER.info(f"Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model")
            torch.save(
                {"model": model.state_dict(), "preds": preds},  # モデルの状態と予測値を保存
                OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth"
            )

    # 保存した最良モデルをロード
    check_point = torch.load(OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth")

    # 検証データに対する予測値を格納
    valid_folds["preds"] = check_point["preds"]

    return valid_folds  # 検証データを含むデータフレームを返す


In [19]:
# モデルの予測結果を基にFβスコアを計算してログに記録する関数。
def get_result(result_df):

    # 予測値を取得
    preds = result_df["preds"].values  # モデルの予測値（確率スコア）

    # 正解ラベルを取得
    labels = result_df["judgement"].values  # 検証データの正解ラベル

    # Fβスコアを計算（閾値で二値化）
    score = fbeta_score(labels, np.where(preds < border, 0, 1), beta=7.0)

    # スコアをログに記録
    LOGGER.info(f"Score: {score:<.5f}")  # 小数点以下5桁まで表示

In [20]:
# モデルの学習、交差検証、推論、結果保存を行うメイン関数。
def main():

    # Training (5-fold クロスバリデーション)
    oof_df = pd.DataFrame()  # Out-of-Fold (OOF) 結果を保存するデータフレーム
    for fold in range(5):
        # 各フォールドでモデルを学習・評価し、結果を取得
        _oof_df = train_loop(train, fold)
        # 全フォールドの結果を結合
        oof_df = pd.concat([oof_df, _oof_df])

        # 各フォールドの結果をログに記録
        LOGGER.info(f"========== fold: {fold} result ==========")
        get_result(_oof_df)  # 各フォールドの評価スコアを計算してログ出力

    # CV result (交差検証の結果)
    LOGGER.info(f"========== CV ==========")
    get_result(oof_df)  # 全体の交差検証スコアを計算してログ出力

    # Save OOF result (OOF結果の保存)
    oof_df.to_csv(OUTPUT_DIR + "oof_df.csv", index=False)  # OOF結果をCSVファイルとして保存

    # Inference (テストデータに対する推論)
    predictions = inference()  # 推論を実行し、予測値を取得
    predictions = np.where(predictions < border, 0, 1)  # 閾値で二値化

    # Submission (提出用ファイルの作成)
    sub["judgement"] = predictions  # テストデータの予測値を設定
    sub.to_csv(OUTPUT_DIR + "submission.csv", index=False, header=False)  # 提出ファイルをCSV形式で保存


In [21]:
if __name__ == "__main__":
    main()

========== fold: 0 training ==========
INFO:__main__:========== fold: 0 training ==========


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

  0%|          | 0/2553 [00:00<?, ?it/s]

========== model: bert-base-uncased fold: 3 inference ==========
INFO:__main__:========== model: bert-base-uncased fold: 3 inference ==========
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2553 [00:00<?, ?it/s]

========== model: bert-base-uncased fold: 4 inference ==========
INFO:__main__:========== model: bert-base-uncased fold: 4 inference ==========
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2553 [00:00<?, ?it/s]